In [16]:
import math

EMISSION_COST = 1.4  # f_c
FRICTION_FACTOR = 0.2  # k
ENGINE_SPEED = 33  # N
ENGINE_DISPLACEMENT = 5  # V
LAMBDA = 1 / (44 * 737)  # xi/(kappa*phi), xi = 1, kappa = 44, phi = 737
CURB_WEIGHT = 6350  # w
GAMMA = 1 / (1000 * 0.4 * 0.9)  # 1/(1000*n_tf*n)
BETTA = 0.5 * 0.7 * 1.2041 * 3.912  # 0.5*Cd*Rho*A
DRIVER_COST = 0.0022  # f_d (pounds/s)
ROLLING_RESISTANCE = 0.01  # C_r
GRAVITY = 9.81  # g
fixed_degree = 0.0107459922
ALPHA = GRAVITY * (math.sin(fixed_degree) + ROLLING_RESISTANCE * math.cos(fixed_degree))




In [17]:
class PRProblem:
    def __init__(self, n_customers, vehicle_curb, max_payload, min_speed, max_speed, dist, customers,
                 fleet_size=2):
        self.n_customers = n_customers
        self.vehicle_curb = vehicle_curb
        self.max_payload = max_payload
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.dist = dist
        self.customers = customers
        self.fleet_size = fleet_size

    def __str__(self):
        return "n. customers: {}\n" \
               "vehicle curb: {}\n" \
               "max payload: {}\n" \
               "min speed: {} max speed: {}\n" \
               "distances: {}\n" \
               "customers: {}".format(self.n_customers,
                                      self.vehicle_curb,
                                      self.max_payload,
                                      self.min_speed, self.max_speed,
                                      self.dist,
                                      self.customers)


In [18]:


def read_instance(inst_name):
    with open("instances/{}.txt".format(inst_name)) as file:
        n_customers = int(file.readline())
        file.readline()
        curb_payload = file.readline().split()
        vehicle_curb = float(curb_payload[0])
        max_payload = float(curb_payload[1])
        file.readline()
        speeds = file.readline().split()
        min_speed = int(speeds[0])
        max_speed = int(speeds[1])
        file.readline()
        dist = {}
        for row in range(0, n_customers+2):
            line = file.readline()
            for col, value in enumerate(line.split()):
                dist[(int(row), int(col))] = float(value)

        file.readline()
        customers = []
        while len(customers) != n_customers+1:
            line = file.readline()
            info = line.split()
            if len(info) > 0:
                customers.append({"id": int(info[0]),
                                  "city_name": info[1],
                                  "demand": float(info[2]),
                                  "ready_time": float(info[3]),
                                  "due_time": float(info[4]),
                                  "service_time": float(info[5])})

    return PRProblem(n_customers, vehicle_curb, max_payload, min_speed, max_speed, dist, customers)


In [19]:
import copy
import time
import random
from random import randrange
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

SEPARATOR = -1


class PRP_Genetic:
    def __init__(self, prp: PRProblem):
        self.prp = prp

    def generate_chromosome(self):
        """
        Generate a random chromosome
        :return: chromosome
        """
        route = []
        for i in range(1, self.prp.n_customers + 1):
            customer = i
            speed = random.randint(self.prp.min_speed, self.prp.max_speed)
            route.append([customer, speed])
        random.shuffle(route)
        routes = self.split_route(route)

        chromosome = []
        for route in routes:
            speed_sep = random.randint(self.prp.min_speed, self.prp.max_speed)
            chromosome += route + [[SEPARATOR, speed_sep]]

        return chromosome

    def generate_initial_population(self, population_size):
        """
        Generate a population with ``population_size`` individuals
        :param population_size: number of individuals of the population being generated
        :return: new population
        """
        return [self.generate_chromosome() for _ in range(population_size)]

    def generate_next_population(self, current_population, crossover_rate, route_mutation_rate, speed_mutation_rate,
                                 selection_method, elitism_rate=0.0, maintain_diversity=False):
        """

        :param current_population: the current population
        :param n_chromosomes: the number of individuals selected from tournament
        :param k_tournament: the number of individuals involved in the tournament
        :param route_mutation_rate: route mutation rate, a real number in the interval [0,1]
        :param speed_mutation_rate:  speed mutation rate, a real number in the interval [0,1]
        :return: new population
        """
        n_parents = len(current_population)
        new_children = []

        n_elitism = round(elitism_rate*n_parents)
        best_chromosomes = sorted(current_population, key=self.fitness)[:n_elitism]
        new_children.extend(best_chromosomes)
        n_parents -= n_elitism
        n_parents = (n_parents if n_parents % 2 == 0 else n_parents - 1)

        parents = selection_method(current_population, n_parents)
        random.shuffle(parents)
        for i in range(0, n_parents, 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]
            if maintain_diversity:
                is_new = False
                while not is_new:
                    child1, child2 = self.mate(parent1, parent2, crossover_rate, route_mutation_rate, speed_mutation_rate)
                    curr_pop = new_children + parents
                    if child1 not in curr_pop and child2 not in curr_pop:
                        is_new = True
            else:
                child1, child2 = self.mate(parent1, parent2, crossover_rate, route_mutation_rate, speed_mutation_rate)

            new_children.append(child1)
            new_children.append(child2)

        return new_children

    def mate(self, parent1, parent2, crossover_rate, route_mutation_rate, speed_mutation_rate):
        parent1 = copy.deepcopy(parent1)
        parent2 = copy.deepcopy(parent2)

        # remove separators, turning the VRP representation into TSP representation
        parent1 = [x for x in parent1 if x[0] != SEPARATOR]
        parent2 = [x for x in parent2 if x[0] != SEPARATOR]

        if random.random() < crossover_rate:
            child1, child2 = self.crossover(parent1, parent2)
        else:
            child1, child2 = parent1, parent2

        child1 = self.select(child1, mutation_rate=route_mutation_rate, speed_mutation_rate=speed_mutation_rate)
        child2 = self.select(child2, mutation_rate=route_mutation_rate, speed_mutation_rate=speed_mutation_rate)
        child1 = self.add_separator(child1)
        child2 = self.add_separator(child2)
        return child1, child2

    def sus(self, population, n, k):
        """
        Stochastic Universal Sampling.
        Reference: https://en.wikipedia.org/wiki/Stochastic_universal_sampling

        :param population: population: current population
        :param k: the number of individuals to selected
        :param n: the number of individuals to keep
        :return: a list of selected individuals
        """
        new_population = []
        n_rounds = round(n/k)
        for _ in range(n_rounds):
            fitness_scores = [1/self.fitness(chromosome) for chromosome in population]
            total_fitness = sum(fitness_scores)
            point_distance = total_fitness / k
            start_point = random.uniform(0, point_distance)
            points = [start_point + i * point_distance for i in range(k)]

            keep = []
            for p in points:
                i = -1
                cumulative_sum = 0
                while cumulative_sum < p:
                    i += 1
                    cumulative_sum += fitness_scores[i]

                keep.append(population[i])
            new_population.extend(keep)
        return new_population

    def tournament_selection(self, population, n, k=2):
        """
        Given a population, make k individuals fight against each other and the one with lowest fitness score wins.
        Repeat this process n times.

        :param population: current population
        :param n: the number of individuals selected from the tournament
        :param k: the number of  individuals involved in the tournament
        :return: a list with the winners
        """
        winners = []
        for _ in range(n):
            elements = random.sample(population, k)
            winners.append(min(elements, key=self.fitness))

        return winners

    def fitness(self, chromosome):
        """

        :param chromosome: a chromosome
        :return: its fitness score
        """
        objective = 0
        prev_customer = 0
        const1 = FRICTION_FACTOR * ENGINE_SPEED * ENGINE_DISPLACEMENT * LAMBDA
        const2 = CURB_WEIGHT * GAMMA * LAMBDA * ALPHA
        const3 = GAMMA * LAMBDA * ALPHA
        const4 = BETTA * GAMMA * LAMBDA

        cumulative_time = 0
        cumulative_payload = 0
        total_payloads = self.compute_total_payload(chromosome)
        route_index = 0
        for item in chromosome:
            current_customer = item[0]
            is_separator = current_customer == SEPARATOR
            if is_separator:
                current_customer = 0
            speed = item[1]
            dist = self.prp.dist[prev_customer, current_customer]
            objective += const1 * dist / speed
            objective += const2 * dist
            objective += const4 * dist * (speed ** 2)

            # Driver Cost
            prev_service_time = self.prp.customers[prev_customer]['service_time']
            displacement_time = self.prp.dist[prev_customer, current_customer] / speed
            total_time = cumulative_time + prev_service_time + displacement_time
            ready_time = self.prp.customers[current_customer]['ready_time']
            if total_time < ready_time:
                total_time = ready_time
            cumulative_time = total_time

            if is_separator:
                objective += DRIVER_COST * cumulative_time
                cumulative_time = 0
                cumulative_payload = 0
                prev_customer = 0
                route_index += 1
            else:
                objective += const3 * dist * (total_payloads[route_index] - cumulative_payload)
                cumulative_payload += self.prp.customers[current_customer]['demand']
                prev_customer = current_customer

        return objective

    def split_route(self, chromosome):
        """
        Given a chromosome, compute the routes according to constraints of payload and time.
        :param chromosome: a chromosome without separator
        :return: a list of routes
        """
        not_visited = copy.deepcopy(chromosome)
        routes = []
        max_payload = self.prp.max_payload
        while len(not_visited) > 0:
            not_visited_aux = copy.deepcopy(not_visited)
            current_route = []
            current_payload = 0
            current_time = 0
            prev_customer = 0
            for item in not_visited_aux:
                current_customer = item[0]
                speed = item[1]

                prev_service_time = self.prp.customers[prev_customer]['service_time']
                displacement_time = self.prp.dist[prev_customer, current_customer] / speed
                total_time = current_time + prev_service_time + displacement_time

                ready_time = self.prp.customers[current_customer]['ready_time']
                due_time = self.prp.customers[current_customer]['due_time']
                if total_time < ready_time:
                    total_time = ready_time

                if (current_payload + self.prp.customers[current_customer]['demand'] <= max_payload) and \
                        (ready_time <= total_time <= due_time):
                    current_payload += self.prp.customers[current_customer]['demand']
                    current_time = total_time
                    not_visited.remove(item)
                    current_route.append(item)  # item is a pair [customer, speed]
                    prev_customer = current_customer
            routes.append(current_route)
        return routes

    
    def select(self, chromosome, mutation_rate, speed_mutation_rate):
       
        _chromosome = copy.deepcopy(chromosome)
        # remove separators
        _chromosome = [x for x in _chromosome if x[0] != SEPARATOR]

        n = len(chromosome)
        for i in range(n):
            if random.random() < speed_mutation_rate:
                _chromosome[i][1] = random.randint(self.prp.min_speed, self.prp.max_speed)

            if random.random() < mutation_rate:
                start = randrange(0, 1)
                end = randrange(start, 1)

                chromosome_mid = chromosome[start:end]
                #chromosome_mid.reverse()
                _chromosome = chromosome[0:start] + chromosome_mid + chromosome[end:] 
        return _chromosome


    def add_separator(self, chromosome):
        chromosome = copy.deepcopy(chromosome)
        # TODO: Caso ocorrer uma mutação, atribuimos valores aleatórios para a velocidade do separador
        routes = self.split_route(chromosome)
        chromosome = []
        for route in routes:
            speed_sep = random.randint(self.prp.min_speed, self.prp.max_speed)
            chromosome += route + [[SEPARATOR, speed_sep]]
        return chromosome

    def crossover(self, parent1, parent2):
        """
        OX crossover: ``Given two parents, two random crossover points are selected partitioning them into a left, middle
        and right portion. The ordered two-point crossover behaves in the following way: child1 inherits its left and
        right section from parent1, and its middle section is determined´´.
        (Accessed from https://www.researchgate.net/publication/331580514_Immune_Based_Genetic_Algorithm_to_Solve_a_Combinatorial_Optimization_Problem_Application_to_Traveling_Salesman_Problem_Volume_5_Advanced_Intelligent_Systems_for_Computing_Sciences)

        Ex.
        Parent1: [9,2,7,|5,4,3,|6,1,8]      Parent1: [9,2,7,|5,4,3,|6,1,8]
        Child1:  [2,8,6,|5,4,3,|9,7,1]      Child2:  [2,7,4,|6,9,5,|3,1,8]
        Parent2: [2,8,3,|6,9,5,|7,4,1]      Parent2: [2,8,3,|6,9,5,|7,4,1]

        After OX Crossover, we select randomly positions to separators

        :param parent1: array representing a route
        :param parent2: array representing a route
        :return: the newly generated children
        """
        route_size = len(parent1)
        parent1_aux = copy.deepcopy(parent1)
        parent2_aux = copy.deepcopy(parent2)

        n = len(parent1)
        index1 = randrange(0, n)
        index2 = randrange(index1, n)

        child1 = [0] * route_size
        child2 = [0] * route_size

        for i in range(index1, index2):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
            for j in range(0, len(parent1_aux)):
                if parent1_aux[j][0] == parent2[i][0]:
                    parent1_aux.remove(parent1_aux[j])
                    break
            for j in range(0, len(parent2_aux)):
                if parent2_aux[j][0] == parent1[i][0]:
                    parent2_aux.remove(parent2_aux[j])
                    break

        for i in range(0, len(parent1_aux)):
            if i < index1:
                child1[i] = parent2_aux[i]
                child2[i] = parent1_aux[i]
                self.change_speed(child1[i], child2[i])
            else:
                child1[i + index2 - index1] = parent2_aux[i]
                child2[i + index2 - index1] = parent1_aux[i]
                self.change_speed(child1[i + index2 - index1], child2[i + index2 - index1])

        return child1, child2

    def change_speed(self, gene1, gene2):
        """
        Given two genes, change their speeds with probability 0.5
        :param gene1:
        :param gene2:
        :return:
        """
        if random.random() < 0.5:
            aux = gene1[1]
            gene1[1] = gene2[1]
            gene2[1] = aux

    def compute_total_payload(self, chromosome):
        """
        Given a chromosome, compute the total payload of each route
        :param chromosome: a chromosome
        :return: a list with the total payload of each route
        """
        total_payloads = []
        current_payload = 0
        for item in chromosome:
            customer = item[0]
            if customer == SEPARATOR:
                total_payloads.append(current_payload)
                current_payload = 0
            else:
                current_payload += self.prp.customers[customer]['demand']

        return total_payloads


def genetic_algorithm_solver(prp, population_size, ngen,
                             crossover_rate_decay,
                             mutation_rate_decay,
                             speed_mutation_rate,
                             selection_method,
                             min_route_mutation_rate=0.0,
                             route_mutation_rate=1.0,
                             time_limit=1800,
                             maintain_diversity=False,
                             elitism_rate=0.0,
                             patience=200):
    population = prp.generate_initial_population(population_size)

    start = time.time()
    fitness_scores = []
    crossover_rate = 1
    non_improvement = 0
    prev_fitness = -1
    for n in range(0, ngen):
        best_chromosome = min(population, key=prp.fitness)
        fitness_score = prp.fitness(best_chromosome)
        fitness_scores.append(fitness_score)
        print("Generation", n, "Best Fitness", fitness_score)
        print("Crossover Rate", crossover_rate, "Mutation Rate", route_mutation_rate)
        population = prp.generate_next_population(current_population=population,
                                                  crossover_rate=crossover_rate,
                                                  route_mutation_rate=route_mutation_rate,
                                                  speed_mutation_rate=speed_mutation_rate,
                                                  elitism_rate=elitism_rate,
                                                  maintain_diversity=maintain_diversity,
                                                  selection_method=selection_method)

        if route_mutation_rate > min_route_mutation_rate:
            route_mutation_rate -= mutation_rate_decay

        crossover_rate *= crossover_rate_decay

        # Time Limit
        end = time.time()
        if (end - start) >= time_limit:
            print("Time Limit", time_limit)
            break

        if (prev_fitness - fitness_score) < 0.1:
            non_improvement += 1
        else:
            non_improvement = 0

        if non_improvement == patience:
            print("Non improvement", non_improvement)
            break
        prev_fitness = fitness_score

    return fitness_scores, best_chromosome


def plot_metrics(scores, filepath=None):
    formatted_dict = {'Generations': [],
                      'Fitness Scores': [], }

    for i, score in enumerate(scores):
        formatted_dict['Generations'].append(i)
        formatted_dict['Fitness Scores'].append(score)

    df_metrics = pd.DataFrame(formatted_dict)
    sns.lineplot(data=df_metrics, x='Generations', y='Fitness Scores')
    if filepath is None:
        plt.show()
    else:
        plt.savefig(filepath)




In [20]:
import random
import time
import pandas as pd


if __name__ == '__main__':
    random.seed(42)


    for instance_size in [100]:
        # for ref_cost in [578.7815538, 643.0906153]:
        for ref_cost in [2175.292804, 2619.790834]:
            results = {'time': [], 'result': [], 'fleet size': []}
            for i in range(1, 150):
                instance_name = "UK{}_01".format(instance_size)
                #config T7
                instance = read_instance(inst_name=instance_name)
                prp = PRP_Genetic(instance)
                ngen = 1000
                final_mutation_rate = 1 / instance_size
                mutation_rate_decay = (1 - final_mutation_rate) / ngen
                start = time.time()
                fitness_scores, best_chromosome = genetic_algorithm_solver(prp,
                                                                           population_size=100,
                                                                           ngen=ngen,
                                                                           crossover_rate_decay=1,
                                                                           mutation_rate_decay=mutation_rate_decay,
                                                                           route_mutation_rate=1,
                                                                           min_route_mutation_rate=1 / instance_size,
                                                                           speed_mutation_rate=1 / instance_size,
                                                                           elitism_rate=0.15,
                                                                           maintain_diversity=False,
                                                                           selection_method=prp.tournament_selection,
                                                                           patience=1000,
                                                                           time_limit=300,
                                                                           #ref_cost=ref_cost
                                                                           )

                n_routes = sum([1 for item in best_chromosome if item[0] == -1])
                print(best_chromosome)
                plot_metrics(fitness_scores)

                end = time.time()
                min_fit = min(fitness_scores)

                results['result'].append(min_fit)
                #results['time'].append(target_time)
                results['fleet size'].append(n_routes)

            df = pd.DataFrame(results)
            df.to_csv('ttplots_uk_{}_configB_{}.csv'.format(instance_size, ref_cost))




Generation 0 Best Fitness 5704.8528517509885
Crossover Rate 1 Mutation Rate 1
Generation 1 Best Fitness 5502.052315940512
Crossover Rate 1 Mutation Rate 0.99901
Generation 2 Best Fitness 5288.7914732342815
Crossover Rate 1 Mutation Rate 0.9980199999999999
Generation 3 Best Fitness 5288.7914732342815
Crossover Rate 1 Mutation Rate 0.9970299999999999
Generation 4 Best Fitness 5162.5034631073095
Crossover Rate 1 Mutation Rate 0.9960399999999998
Generation 5 Best Fitness 5162.5034631073095
Crossover Rate 1 Mutation Rate 0.9950499999999998
Generation 6 Best Fitness 5109.706609390109
Crossover Rate 1 Mutation Rate 0.9940599999999997
Generation 7 Best Fitness 5002.226464495837
Crossover Rate 1 Mutation Rate 0.9930699999999997
Generation 8 Best Fitness 4737.032391774581
Crossover Rate 1 Mutation Rate 0.9920799999999996
Generation 9 Best Fitness 4737.032391774581
Crossover Rate 1 Mutation Rate 0.9910899999999996
Generation 10 Best Fitness 4588.20727382693
Crossover Rate 1 Mutation Rate 0.990099

In [ ]:
"""
this model is based on paper "An adaptive large neighborhood search heuristic for the
Pollution-Routing Problem", Emrah Demir, Tolga Bektas, Gilbert Laporte
"""
"""import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import random
import math

K = 10**9
L = 10**9

def compute_objective(instance:PRProblem, x_vars, z_vars, f_vars, s_vars):
    const1 = FRICTION_FACTOR * ENGINE_SPEED * ENGINE_DISPLACEMENT * LAMBDA
    const2 = CURB_WEIGHT * GAMMA * LAMBDA
    const3 = GAMMA * LAMBDA
    const4 = BETTA * GAMMA * LAMBDA
    alphas = genarateRandomAlphaMatrix()

    objective = 0
    for i in range(0, len(instance.customers)):
        for j in range(0, len(instance.customers)):
            if i != j:
                sum_z = 0
                sum_z2 = 0
                for r in range(instance.min_speed, instance.max_speed + 1):
                    sum_z += z_vars[(i, j, r)] / r
                    sum_z2 += z_vars[(i, j, r)] * (r ** 2)
                objective += const1 * instance.dist[(i, j)] * sum_z
                objective += const2 * ALPHA * instance.dist[(i, j)] * x_vars[(i, j)]
                objective += const3 * ALPHA * instance.dist[(i, j)] * f_vars[(i, j)]
                objective += const4 * instance.dist[(i, j)] * sum_z2
        if i > 0:
            objective += DRIVER_COST * s_vars[i]
    return objective


def constraint_12(model, instance:PRProblem, x_vars):
    _sum = 0
    for j in range(1, len(instance.customers)):
        _sum += x_vars[(0, j)]
    
    constraint_12_name = "constraint_12"
    model.addConstr(_sum == instance.fleet_size, constraint_12_name)


def constraint_13(model, instance:PRProblem, x_vars):
    for i in range(1, len(instance.customers)):
        _sum = 0
        for j in range(0, len(instance.customers)):
            if i != j:
                _sum += x_vars[(i, j)]
        
        constraint_13_name = "constraint_13_{}".format(i)
        model.addConstr(_sum == 1, constraint_13_name)


def constraint_14(model, instance: PRProblem, x_vars):
    for j in range(1, len(instance.customers)):
        _sum = 0
        for i in range(0, len(instance.customers)):
            if i != j:
                _sum += x_vars[(i, j)]

        constraint_14_name = "constraint_14_{}".format(j)
        model.addConstr(_sum == 1, constraint_14_name)


def constraint_15(model, instance:PRProblem, f_vars):
    for i in range(1, len(instance.customers)):
        q = instance.customers[i]["demand"]
        _sum1 = 0
        _sum2 = 0
        for j in range(0, len(instance.customers)):
            if i != j:
                _sum1 += f_vars[(j, i)]
                _sum2 += f_vars[(i, j)]

        constraint_15_name = "constraint_15_{}".format(i)
        model.addConstr(_sum1 - _sum2 == q, constraint_15_name)


def constraint_16(model, instance: PRProblem,  x_vars, f_vars):
    Q = instance.max_payload
    for i in range(0, len(instance.customers)):
        q_i = instance.customers[i]["demand"]
        for j in range(0, len(instance.customers)):
            q_j = instance.customers[j]["demand"]
            if i != j:
                constraint_16_1 = "constraint_16_1_{}_{}".format(i, j)
                constraint_16_2 = "constraint_16_2_{}_{}".format(i, j)
                model.addConstr(q_j * x_vars[(i, j)] <= f_vars[(i, j)], constraint_16_1)
                model.addConstr(f_vars[(i, j)] <= (Q - q_i) * x_vars[(i, j)], constraint_16_2)


def constraint_17(model, instance: PRProblem,  y_vars, z_vars, x_vars):
    for i in range(0, len(instance.customers)):
        t_i = instance.customers[i]["service_time"]
        for j in range(1, len(instance.customers)):
            _sum = 0
            if i != j:
                if i > 0:
                    _sum = y_vars[i] - y_vars[j] + t_i
                else:
                    _sum = - y_vars[j] + t_i

                for r in range(instance.min_speed, instance.max_speed + 1):
                    _sum += instance.dist[(i, j)] * z_vars[(i, j, r)] / r   

                constraint_17_name = "constraint_17_{}_{}".format(i, j)
                model.addConstr(_sum <= K * (1 - x_vars[(i, j)]), constraint_17_name)

def constraint_17_indicator(model, instance: PRProblem,  y_vars, z_vars, x_vars):
    # reference: https://www.gurobi.com/documentation/8.0/refman/py_model_addgenconstrindic.html

    for i in range(0, len(instance.customers)):
        t_i = instance.customers[i]["service_time"]
        for j in range(1, len(instance.customers)):
            _sum = 0
            if i != j:
                if i > 0:
                    _sum = y_vars[i] + t_i
                else:
                    _sum = t_i
                for r in range(instance.min_speed, instance.max_speed + 1):
                    _sum += instance.dist[(i, j)] * z_vars[(i, j, r)] / r

                model.addGenConstrIndicator(x_vars[(i, j)], True, _sum <= y_vars[j])

def constraint_18(model, instance: PRProblem,  y_vars):
    for i in range(1, len(instance.customers)):
        a = instance.customers[i]["ready_time"]
        b = instance.customers[i]["due_time"]

        constraint_id_18_1 = "constraint_18_1_{}".format(i)
        constraint_id_18_2 = "constraint_18_2_{}".format(i)
        model.addConstr(a <= y_vars[i], constraint_id_18_1)
        model.addConstr(y_vars[i] <= b, constraint_id_18_2)


def constraint_19(model, instance: PRProblem,  x_vars, z_vars, y_vars, s_vars):
    for j in range(1, len(instance.customers)):
        t_j = instance.customers[j]["service_time"]
        _sum = y_vars[j] + t_j - s_vars[j]
        for r in range(instance.min_speed, instance.max_speed + 1):
            _sum += instance.dist[(j, 0)] * z_vars[(j, 0, r)] / r

        constraint_19_name = "constraint_19_{}".format(j)
        model.addConstr(_sum <= L * (1 - x_vars[(j, 0)]), constraint_19_name)


def constraint_19_indicator(model, instance: PRProblem,  x_vars, z_vars, y_vars, s_vars):
    for j in range(1, len(instance.customers)):
        t_j = instance.customers[j]["service_time"]
        _sum = y_vars[j] + t_j
        for r in range(instance.min_speed, instance.max_speed + 1):
            _sum += instance.dist[(j, 0)] * z_vars[(j, 0, r)] / r

        model.addGenConstrIndicator(x_vars[(j, 0)], True, _sum <= s_vars[j])


def constraint_20(model, instance:PRProblem, x_vars, z_vars):
    for i in range(0, len(instance.customers)):
        for j in range(0, len(instance.customers)):
            if i != j:
                _sum = 0
                for r in range(instance.min_speed, instance.max_speed + 1):
                    _sum += z_vars[(i, j, r)]

                constraint_20_name = "constraint_20_{}_{}".format(i, j)
                model.addConstr(_sum == x_vars[(i, j)], constraint_20_name)


def build_model(instance: PRProblem):
    model = gp.Model("PRP")

    z_vars = {}
    x_vars = {}
    f_vars = {}
    y_vars = {}
    s_vars = {}

    for i in range(0, len(instance.customers)):
        if i > 0:
            y_vars[i] = model.addVar(vtype=GRB.CONTINUOUS, lb=0, name="y_{}".format(i))
            s_vars[i] = model.addVar(vtype=GRB.CONTINUOUS, lb=0, name="s_{}".format(i))
        for j in range(0, len(instance.customers)):
            if i != j:
                x_vars[(i, j)] = model.addVar(vtype=GRB.BINARY, name="x_{}_{}".format(i, j))
                f_vars[(i, j)] = model.addVar(vtype=GRB.CONTINUOUS, lb=0, name="f_{}_{}".format(i, j))
                for r in range(instance.min_speed, instance.max_speed + 1):
                    z_vars[(i, j, r)] = model.addVar(vtype=GRB.BINARY, name="z_{}_{}_{}".format(i, j, r))
    instance.fleet_size = model.addVar(vtype=GRB.INTEGER, lb=0, name="m")
    objective = compute_objective(instance, x_vars, z_vars, f_vars, s_vars)
    model.setObjective(objective, GRB.MINIMIZE)

    # adding constraints
    constraint_12(model, instance, x_vars)
    constraint_13(model, instance, x_vars)
    constraint_14(model, instance, x_vars)
    constraint_15(model, instance, f_vars)
    constraint_16(model, instance,  x_vars, f_vars)
    # constraint_17(model, instance,  y_vars, z_vars, x_vars)
    constraint_17_indicator(model, instance, y_vars, z_vars, x_vars)
    constraint_18(model, instance,  y_vars)
    # constraint_19(model, instance, x_vars, z_vars, y_vars, s_vars)
    constraint_19_indicator(model, instance, x_vars, z_vars, y_vars, s_vars)
    constraint_20(model, instance, x_vars, z_vars)

    return model, x_vars, z_vars

def genarateRandomAlphaMatrix():
    alphas = {}

    for i in range(0, len(instance.customers)):
        for j in range(0, len(instance.customers)):
            fixedDegree = 0.0107459922
            alphas[(i, j)] = GRAVITY * (math.sin(fixedDegree) + ROLLING_RESISTANCE * math.cos(fixedDegree))

    return alphas

def save_results_CSV(lower_bounds:list, upper_bounds:list, times:list, fleet_sizes:list, filepath):
    df = pd.DataFrame({'time':times, 'lower_bounds':lower_bounds, 'upper_bounds':upper_bounds, 'fleet size':fleet_sizes})
    df.to_csv(filepath)

if __name__ == '__main__':
    random.seed(42)
    lower_bounds = []
    upper_bounds = []
    times = []
    fleet_sizes = []
    for n in [100]:
        for i in range(1, 2):
            if i < 10:
                instance_name = "{}{}".format("UK{}_0".format(n), i)
            else:
                instance_name = "{}{}".format("UK{}_".format(n), i)
            print(instance_name)
            instance = read_instance(inst_name=instance_name)

            model, x_vars, z_vars = build_model(instance)
            model.setParam('TimeLimit', 1800)
            model.optimize()
            print('Final Objective: {}'.format(model.objVal))
            for v in model.getVars():
                if v.varName == "m":
                    print('Fleet size: %g' % (v.x))
                    fleet_sizes.append(v.x)

            print('Route')
            for edge, v in x_vars.items():
                if v.X > 0.9:
                    for r in range(instance.min_speed, instance.max_speed + 1):
                        if z_vars[(edge[0], edge[1], r)].X > 0.9:
                            print("{}: {}".format(v.varName, v.X), "velocidade:", r)

            lower_bounds.append(model.objBound)
            upper_bounds.append(model.objVal)
            times.append(model.runtime)"""

        # filepath = "./pl_{}_results.csv".format(n)
        # save_results_CSV(lower_bounds, upper_bounds, times, fleet_sizes, filepath)

In [ ]:
"""import random
import time
import pandas as pd

def sus_A(elitism, k):
    results = {'time': [], 'result': [], 'fleet size': []}
    for i in range(1, 21):
        if i < 10:
            instance_name = "{}{}".format("UK{}_0".format(instance_size), i)
        else:
            instance_name = "{}{}".format("UK{}_".format(instance_size), i)

        instance = read_instance(inst_name=instance_name)
        prp = PRP_Genetic(instance)
        ngen = 100000
        start = time.time()
        fitness_scores, best_chromosome = genetic_algorithm_solver(prp,
                                                                   population_size=100,
                                                                   ngen=ngen,
                                                                   crossover_rate_decay=1,
                                                                   mutation_rate_decay=0,
                                                                   route_mutation_rate=1 / instance_size,
                                                                   min_route_mutation_rate=1 / instance_size,
                                                                   speed_mutation_rate=1 / instance_size,
                                                                   elitism_rate=elitism,
                                                                   maintain_diversity=False,
                                                                   selection_method=lambda pop, n: prp.sus(pop, n, k=k))
        n_routes = sum([1 for item in best_chromosome if item[0] == -1])
        print(best_chromosome)
        # plot_metrics(fitness_scores)

        end = time.time()
        min_fit = min(fitness_scores)

        results['result'].append(min_fit)
        results['time'].append((end - start))
        results['fleet size'].append(n_routes)

    return results

def sus_B(elitism, k):
    results = {'time': [], 'result': [], 'fleet size': []}
    for i in range(1, 21):
        if i < 10:
            instance_name = "{}{}".format("UK{}_0".format(instance_size), i)
        else:
            instance_name = "{}{}".format("UK{}_".format(instance_size), i)

        instance = read_instance(inst_name=instance_name)
        prp = PRP_Genetic(instance)
        ngen = 100000
        final_mutation_rate = 1 / instance_size
        mutation_rate_decay = (1 - final_mutation_rate) / ngen
        start = time.time()
        fitness_scores, best_chromosome = genetic_algorithm_solver(prp,
                                                                   population_size=100,
                                                                   ngen=ngen,
                                                                   crossover_rate_decay=1,
                                                                   mutation_rate_decay=mutation_rate_decay,
                                                                   route_mutation_rate=1,
                                                                   min_route_mutation_rate=1 / instance_size,
                                                                   speed_mutation_rate=1 / instance_size,
                                                                   elitism_rate=elitism,
                                                                   maintain_diversity=False,
                                                                   selection_method=lambda pop, n: prp.sus(pop, n, k=k))
        n_routes = sum([1 for item in best_chromosome if item[0] == -1])
        print(best_chromosome)
        # plot_metrics(fitness_scores)

        end = time.time()
        min_fit = min(fitness_scores)

        results['result'].append(min_fit)
        results['time'].append((end - start))
        results['fleet size'].append(n_routes)

    return results


def tournament_A(elitism):
    results = {'time': [], 'result': [], 'fleet size': []}
    for i in range(1, 21):
        if i < 10:
            instance_name = "{}{}".format("UK{}_0".format(instance_size), i)
        else:
            instance_name = "{}{}".format("UK{}_".format(instance_size), i)

        instance = read_instance(inst_name=instance_name)
        prp = PRP_Genetic(instance)
        ngen = 100000
        start = time.time()
        fitness_scores, best_chromosome = genetic_algorithm_solver(prp,
                                                                   population_size=100,
                                                                   ngen=ngen,
                                                                   crossover_rate_decay=1,
                                                                   mutation_rate_decay=0,
                                                                   route_mutation_rate=1 / instance_size,
                                                                   min_route_mutation_rate=1 / instance_size,
                                                                   speed_mutation_rate=1 / instance_size,
                                                                   elitism_rate=elitism,
                                                                   maintain_diversity=False,
                                                                   selection_method=prp.tournament_selection)
        n_routes = sum([1 for item in best_chromosome if item[0] == -1])
        print(best_chromosome)
        # plot_metrics(fitness_scores)

        end = time.time()
        min_fit = min(fitness_scores)

        results['result'].append(min_fit)
        results['time'].append((end - start))
        results['fleet size'].append(n_routes)

    return results


def tournament_B(elitism):
    results = {'time': [], 'result': [], 'fleet size': []}
    for i in range(1, 21):
        if i < 10:
            instance_name = "{}{}".format("UK{}_0".format(instance_size), i)
        else:
            instance_name = "{}{}".format("UK{}_".format(instance_size), i)

        instance = read_instance(inst_name=instance_name)
        prp = PRP_Genetic(instance)
        ngen = 100000
        final_mutation_rate = 1 / instance_size
        mutation_rate_decay = (1 - final_mutation_rate) / ngen
        start = time.time()
        fitness_scores, best_chromosome = genetic_algorithm_solver(prp,
                                                                   population_size=100,
                                                                   ngen=ngen,
                                                                   crossover_rate_decay=1,
                                                                   mutation_rate_decay=mutation_rate_decay,
                                                                   route_mutation_rate=1,
                                                                   min_route_mutation_rate=1 / instance_size,
                                                                   speed_mutation_rate=1 / instance_size,
                                                                   elitism_rate=elitism,
                                                                   maintain_diversity=False,
                                                                   selection_method=prp.tournament_selection)
        n_routes = sum([1 for item in best_chromosome if item[0] == -1])
        print(best_chromosome)

        end = time.time()
        min_fit = min(fitness_scores)

        results['result'].append(min_fit)
        results['time'].append((end - start))
        results['fleet size'].append(n_routes)

    return results


if __name__ == '__main__':
    random.seed(42)

    for elitism in [0.15]:
        for k in [5, 50]:
            for instance_size in [10, 20, 100, 200]:
                print("==================== Instance {}, elitism {}, k {}".format(instance_size, elitism, k))
                results = sus_A(elitism, k)
                df = pd.DataFrame(results)
                df.to_csv('ga_{}_config_A_{}_SUS_{}_results.csv'.format(instance_size, elitism, k))

                results = sus_B(elitism, k)
                df = pd.DataFrame(results)
                df.to_csv('ga_{}_config_B_{}_SUS_{}_results.csv'.format(instance_size, elitism, k))

"""